# AI Medicines Assitince

## Important imports and setup

In [3]:
# Imports
import openai
import os
import requests
from dotenv import load_dotenv
from langchain.agents import initialize_agent, Tool
from langchain.llms import OpenAI as LangChainOpenAI
from langchain.agents.agent_types import AgentType
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory
import tkinter as tk
from tkinter import scrolledtext, filedialog
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wavfile
import tempfile
import pyttsx3
import easyocr
import time
from PIL import Image
from requests.adapters import HTTPAdapter, Retry

# Load environment variables
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")


## 1. Text Preprocessing + Vector Store

In [4]:
# Text Preprocessing
def preprocess_text(text):
    text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    return text_splitter.split_text(text)

# Setup Vector Database
def setup_vector_db(texts):
    embeddings = OpenAIEmbeddings()
    vector_store = FAISS.from_texts(texts, embeddings)
    return vector_store


## 2. OpenFDA with Retry and Timeout

In [5]:
# Requests session with retries
session = requests.Session()
retries = Retry(total=3, backoff_factor=0.5, status_forcelist=[500, 502, 503, 504])
session.mount('https://', HTTPAdapter(max_retries=retries))

# Search OpenFDA
def search_openfda(medicine_name):
    try:
        url = f"https://api.fda.gov/drug/label.json?search=openfda.brand_name:{medicine_name}&limit=1"
        response = session.get(url, timeout=5)
        data = response.json()

        if 'results' in data:
            result = data['results'][0]
            brand_name = result['openfda'].get('brand_name', ['Unknown'])[0]
            manufacturer = result['openfda'].get('manufacturer_name', ['Unknown'])[0]
            usage = result.get('indications_and_usage', ['No usage information'])[0]
            dosage = result.get('dosage_and_administration', ['No dosage information'])[0]
            warnings = result.get('warnings', ['No warnings'])[0]

            return f"""
📋 Drug Information:
- Brand Name: {brand_name}
- Manufacturer: {manufacturer}
- Usage: {usage}
- Dosage: {dosage}
- Warnings: {warnings}
"""
        else:
            return "❌ No information found in OpenFDA."
    except Exception as e:
        return f"Error fetching from OpenFDA: {e}"


## 3. LLM, LangChain Agent, Memory, Tools



In [14]:
# OpenFDA Tool for LangChain
openfda_tool = Tool(
    name="search_openfda",
    func=search_openfda,
    description="Use this tool to get information about medicines from the OpenFDA database."
)

# LLM Setup
llm = LangChainOpenAI(temperature=0, openai_api_key=openai.api_key)

# Conversation Memory
memory = ConversationBufferMemory(memory_key="chat_history")

# Initialize LangChain Agent
agent = initialize_agent(
    tools=[openfda_tool],
    llm=llm,
    agent_type=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)


## 4. Audio Functions

In [15]:
# Recognize Speech
def recognize_speech_openai():
    fs = 16000
    seconds = 5
    print("Recording...")
    myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=1)
    sd.wait()
    print("Recording finished.")

    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as f:
        wavfile.write(f.name, fs, (myrecording * 32767).astype(np.int16))
        with open(f.name, "rb") as audio_file:
            transcript = openai.Audio.transcribe("whisper-1", audio_file)
        return transcript["text"]

# Speak Text
def speak_text(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()


## 5. EasyOCR Image-to-Text

In [16]:
# Extract drug name from image
def extract_drug_name(image_path):
    reader = easyocr.Reader(['en'], gpu=False)
    results = reader.readtext(image_path)

    if not results:
        return None

    texts_with_heights = []
    for bbox, text, confidence in results:
        top_left = bbox[0]
        bottom_left = bbox[3]
        height = abs(bottom_left[1] - top_left[1])
        texts_with_heights.append((text, height))

    largest_text = max(texts_with_heights, key=lambda x: x[1])[0]
    return largest_text.lower().strip()


## 6. GUI Tkinter App

In [17]:
class MedicineAssistantApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Medicine Assistant")
        self.root.geometry("650x650")

        frame = tk.Frame(self.root)
        frame.pack(padx=10, pady=10, expand=True, fill="both")

        self.conversation_display = scrolledtext.ScrolledText(frame, wrap=tk.WORD, width=70, height=30, font=("Arial", 12))
        self.conversation_display.pack(pady=5)
        self.conversation_display.tag_configure("user", foreground="blue")
        self.conversation_display.tag_configure("bot", foreground="green")
        self.conversation_display.config(state=tk.DISABLED)

        self.user_input = tk.Entry(frame, width=70, font=("Arial", 12))
        self.user_input.pack(pady=5)

        self.send_button = tk.Button(frame, text="Send", font=("Arial", 12), command=self.handle_user_input)
        self.send_button.pack(pady=5)

        self.speech_button = tk.Button(frame, text="Speak", font=("Arial", 12), command=self.handle_speech_input)
        self.speech_button.pack(pady=5)

        self.upload_button = tk.Button(frame, text="Upload Image", font=("Arial", 12), command=self.handle_image_upload)
        self.upload_button.pack(pady=5)

        # Example: Initialize vector store (You can load a custom database here)
        self.vector_store = None

    def display_message(self, message, is_user=False):
        self.conversation_display.config(state=tk.NORMAL)
        if is_user:
            self.conversation_display.insert(tk.END, f"You: {message}\n", "user")
        else:
            self.conversation_display.insert(tk.END, f"Bot: {message}\n", "bot")
        self.conversation_display.config(state=tk.DISABLED)
        self.conversation_display.yview(tk.END)

    def handle_user_input(self):
        user_query = self.user_input.get()
        if user_query.lower() in ["exit", "quit"]:
            self.root.quit()

        self.display_message(user_query, is_user=True)

        response = self.process_query(user_query)

        self.display_message(response)
        speak_text(response)
        self.user_input.delete(0, tk.END)

    def process_query(self, query):
        if "what is" in query.lower() or "tell me about" in query.lower():
            medicine_name = query.lower().replace("what is", "").replace("tell me about", "").strip()
            return agent.run(medicine_name)
        else:
            return chat_with_gpt(query)

    def handle_speech_input(self):
        try:
            speech_text = recognize_speech_openai()
            self.display_message(speech_text, is_user=True)
            response = self.process_query(speech_text)
            self.display_message(response)
            speak_text(response)
        except Exception as e:
            self.display_message(f"Speech recognition failed: {e}")
            speak_text("An error occurred while processing speech input.")

    def handle_image_upload(self):
        file_path = filedialog.askopenfilename(
            filetypes=[("Image files", "*.png;*.jpg;*.jpeg;*.bmp")]
        )
        if not file_path:
            return

        self.display_message("Image uploaded. Extracting medicine name...")

        drug_name = extract_drug_name(file_path)

        if drug_name:
            self.display_message(f"Detected Medicine Name: {drug_name}")

            try:
                response = agent.run(drug_name)
                self.display_message(response)
                speak_text(response)
            except Exception as e:
                error_msg = f"Failed to fetch drug info: {e}"
                self.display_message(error_msg)
                speak_text(error_msg)
        else:
            error_msg = "Could not extract medicine name from image."
            self.display_message(error_msg)
            speak_text(error_msg)

# Chat function
def chat_with_gpt(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful medicine assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    return response['choices'][0]['message']['content']

# Run app
if __name__ == "__main__":
    root = tk.Tk()
    app = MedicineAssistantApp(root)
    root.mainloop()


Using CPU. Note: This module is much faster with a GPU.
c:\Users\R\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)




> Entering new AgentExecutor chain...
 I should use the search_openfda tool to get information about medicines
Action: search_openfda
Action Input: 'aspirin'
Observation: 
📋 Drug Information:
- Brand Name: Low Dose Aspirin Enteric Safety-Coated
- Manufacturer: P & L Development, LLC
- Usage: Uses for the temporary relief of minor aches and pains or as recommended by your doctor. Because of its delayed action, this product will not provide fast relief of headaches or other symptoms needing immediate relief. ask your doctor about other uses for safety coated 81 mg aspirin
- Dosage: Directions drink a full glass of water with each dose adults and children 12 years and over: take 4 to 8 tablets every 4 hours not to exceed 48 tablets in 24 hours unless directed by a doctor children under 12 years: consult a doctor
- Warnings: Warnings Reye's syndrome : Children and teenagers who have or are recovering from chicken pox or flu-like symptoms should not use this product. When using this produ